In [ ]:
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F

# 读取标签数据
label_file_path = '/home/myid/jw87501/test/train/Annotations/neck.xlsx'  
df = pd.read_excel(label_file_path, header=None)


image_dir = '/home/myid/jw87501/test/train_image/neck_design_labels'  

# 将图片文件名与路径拼接成完整路径
df[0] = df[0].apply(lambda x: os.path.join(image_dir, x.split('/')[-1]))

# 2. 创建自定义数据集类
class FashionDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0] 
        image = Image.open(img_path).convert("RGB") 
        label = self.dataframe.iloc[idx, 1]  

        # 将标签从字符串形式（如 'nnnnnnmyn'）转为 one-hot 编码
        one_hot_label = [1 if char == 'y' else 0 for char in label]  # 'y' 为 1，其他为 0

        if self.transform:
            image = self.transform(image)
        return image, one_hot_label

# 3. 定义图片预处理操作
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  # 转换为 Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# 4. 创建数据集和数据加载器
train_dataset = FashionDataset(dataframe=df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 测试数据加载器
for images, labels in train_loader:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')
    break  
print(len(train_dataset))

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels: [tensor([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 1]), tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0]), tensor([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])]
5696


In [ ]:
# 读取测试标签数据
test_label_file_path = '/home/myid/jw87501/test/train/Annotations/cleaned_answer.xlsx' 
test_df = pd.read_excel(test_label_file_path, header=None)

# 测试图片所在目录
test_image_dir = '/home/myid/jw87501/test/test_image/neck_design_labels'  


test_df[0] = test_df[0].apply(lambda x: os.path.join(test_image_dir, x.split('/')[-1]))

# 创建测试集数据集类/测试集数据加载器
test_dataset = FashionDataset(dataframe=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
for images, labels in test_loader:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')
    break  
print(len(test_dataset))

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels: [tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0]), tensor([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 0, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0])]
708


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 输入 3 通道，输出 32 通道
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # 输入平铺为 64 个 56x56 的特征图
        self.fc2 = nn.Linear(128, 5)  # 输出 32 类别的预测（根据你的 one-hot 标签长度）
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = x.view(-1, 64 * 56 * 56)  # 展平
        x = F.relu(self.fc1(x)) 
        x = self.dropout(x)
        x = self.fc2(x)  
        return x

# 创建模型、损失函数和优化器
model = CNN().to(device)
criterion = nn.BCEWithLogitsLoss()  # 二元交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
EPOCHS = 1  
losses = []
accs = []

for epoch in range(EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

  
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{EPOCHS}', unit='batch') as pbar:
        model.train()  
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = torch.stack(labels).T.float().to(device)  
            
            optimizer.zero_grad()  
            outputs = model(images)  
            loss = criterion(outputs, labels) 
            
            loss.backward()  
            optimizer.step()  
            
            running_loss += loss.item()
            pbar.set_postfix({'Loss': running_loss / (i + 1)})
            pbar.update(1)

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

print('Finished Training')

# 测试模型
model.eval() 
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():  
    for images, labels in test_loader:  
        images = images.to(device)
        labels = torch.stack(labels).T.float().to(device) 
        
        outputs = model(images)  
        loss = criterion(outputs, labels)  
        test_loss += loss.item()

        predicted = (outputs > 0.5).float()  # 将输出二值化
        total += labels.size(0)  # 累加总样本数
        correct += (predicted == labels).sum().item() 

# 计算平均损失
test_loss /= len(test_loader)

# 计算测试准确率
accuracy = 100 * correct / (total * labels.size(1))  # 正确率：正确预测的样本数 / 总样本数

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')



Epoch 1/1:   0%|          | 0/178 [00:00<?, ?batch/s]